## Gmail Organization with LLM's(Langchain, Ollama, OpenAI)

- [Introduction](#introduction)
- [Prerequisites](#prerequisites)
- [Installing Dependencies](#installing-dependencies)
- [Loading Environment Variables](#loading-environment-variables)
- [Configuring Categories and Actions](#configuring-categories-and-actions)
- [Fetching Emails](#fetching-emails)
- [Using the LLM for Email Categorization](#using-the-llm-for-email-categorization)
- [Performing Actions on Categorized Emails](#performing-actions-on-categorized-emails)
- [Running the Application](#running-the-application)
- [Conclusion and Next Steps](#conclusion-and-next-steps)



<a id = 'intro'></a>
### Introduction
This project is designed to help you efficiently manage your Gmail inbox by leveraging the power of machine learning and automation. With this application, you can automatically categorize your emails into predefined categories and perform various actions such as marking them as read, moving them to specific folders, or even deleting them.

**Key Features:**  
- **Fetch and Process Emails**: Seamlessly connect to your Gmail account and retrieve the latest emails, including important details like the sender, subject, and body.
- **Email Categorization with LLM**: Utilize a language model (LLM) to automatically categorize your emails based on the content, making your inbox more organized.
- **Automated Actions**: Depending on the category assigned to each email, the application can automatically perform actions like marking emails as read, moving them to folders, or deleting them.
- **Customizable Workflow**: You have full control over how emails are categorized and what actions are taken. Simply configure the settings in the provided files to match your specific needs.

**What You'll Learn:**  
In this notebook, you'll learn how to set up and run the application from start to finish. You'll begin by installing the necessary dependencies and setting up environment variables. Then, you'll move on to configuring email categories and actions, fetching and categorizing emails, and finally, performing automated actions on those emails.

By the end of this notebook, you'll have a fully functional email management tool that you can customize and extend according to your preferences. Whether you're looking to clean up your inbox or streamline your email workflow, this application will help you achieve your goals.


<a id = 'prerequisites'></a>
### Prerequisites <a name="prerequisites"></a>

Before getting started with this project, you'll need to ensure that the following components are set up on your system:

1. **Gmail Account with 2-Factor Authentication (2FA):**
   - Make sure 2FA is enabled on your Gmail account to add an extra layer of security.
   - After enabling 2FA, you'll need to generate an app password. This password allows the tool to securely access your Gmail account. You can generate an app password in your Google account settings under "Security", If you can't find it, search for "App Password." Once created, use this as your email password.

2. **Ollama Installed Locally or an OpenAI API Key:**
   - If you prefer using Ollama's LLMs, ensure that Ollama is installed on your local system. Follow the installation instructions provided by Ollama.
   - Alternatively, if you're using OpenAI's models, you'll need an API key from OpenAI. Sign up for an OpenAI account and obtain your API key from the API section.

Once these components are in place, you'll be ready to proceed with setting up and running the application.

 <a name="installing-dependencies"></a>
### Installing Dependencies

To get started, you'll need to install the required Python packages for this project. All necessary dependencies are listed in the `requirements.txt` file.

Simply run the following command in your terminal to install them:

```bash
pip install -r requirements.txt
```

In [ ]:
# pip install -r requirements.txt

<a name="loading-environment-variables"></a>
### Loading Environment Variables

Populate the `dev.env` file with your `email_id`, `email_password`, and `openai_key` (if you plan on using ChatGPT as your model). Once your `dev.env` file is ready, run the code cell below to load the variables:

```python
from dotenv import load_dotenv
load_dotenv('dev.env')
```

This will securely load your environment variables into the application, allowing you to proceed with the rest of the setup.

In [ ]:
from dotenv import load_dotenv
load_dotenv('dev.env')

<a id = 'configuring-categories-and-actions'></a>
### Configuring Categories and Actions
This step involves defining how your emails will be categorized and what actions will be taken based on those categories. The `constants.py` file is where you can customize these settings.

- **Categories:** Define categories for classifying emails, such as "Personal," "Work," "Spam," etc.

```python
categories = {
    "Personal": "Emails from friends, family, and personal contacts.",
    "Social":  "Notifications from social media platforms.",
    ...
}
```

- **Category Actions:** Specify actions like auto-delete, auto-sort, and auto-read for each category.
  - Set `ON` to enable an action and `OFF` to disable it (defaults are recommended)

```python
category_actions = {
    "Personal": {
        "auto-delete": "OFF",  # If set to 'ON', emails belonging to this category will be deleted
        "auto-sort": "ON",     # If set to 'ON', emails will be moved to the determined target folder
        "auto-read": "ON",     # If set to 'ON', emails will be marked as read
        "remove-after-sort": "ON"  # If set to 'ON', emails will be removed from the source folder after being moved to the target folder
    },
    ...
}
```

- **Default Google Labels:** These are the default labels provided by Gmail, such as "INBOX," "SENT," etc. Do not change them.

```python
default_gmail_labels = [
    'INBOX', '[Gmail]/All Mail', '[Gmail]/Bin', '[Gmail]/Drafts', '[Gmail]/Important',
    '[Gmail]/Sent Mail', '[Gmail]/Spam', '[Gmail]/Starred'
]
```

- **Custom Labels:** These are the custom labels you would like to move your emails to. You should create these labels manually in your Gmail account with the exact name and case.
  - Note: The names of your custom labels cannot be "Personal," "Social," "Promotions," etc., as these are reserved by Google. The custom labels should be prefixed with "@" (or any other symbol) to differentiate them from the default labels.

```python
# Custom labels to be created in Gmail
custom_labels = [ 
    "@ Personal", "@ Social", "@ Promotions", "@ Verifications", 
    "@ Finances", "@ Reminders", "@ Work", "@ Support"
]
```

- **Category Mapper:** Use this to control into which folder/label or mailbox the emails should be moved.

```python
category_mapper = {
    "Personal": "@ Personal",  # Move emails of category 'Personal' to '@Personal'
    "Social": "@ Social",      # Move emails of category 'Social' to '@ Social' label
    ...
}
```

**Note**  
- `constants.py` has few functions to check whether categories , category_settings,
  custom_labels, category_mapper have been correctly defined or not
- Donot forget to create custom labels listed in `custom_labels` in gmail with same name 
  and case.

In [ ]:
# Load constants and utility functions
from constants import *
from utils import *

<a id = 'fetching-emails'></a>
### Fetching Emails
In this section, we'll cover the essential functions needed to connect to the mail server, list and validate mailboxes (folders, labels), and fetch emails. These functions form the backbone of the email management system, enabling the application to interact with your Gmail account and retrieve the necessary data for processing.

In [ ]:

import os
import imaplib
from tqdm import tqdm
import chardet
import email
from email.header import decode_header
import pandas as pd


# Connect to the mail server
def mail_session():
    email_id = os.environ.get('MAIL_USERNAME')
    email_pass = os.environ.get('MAIL_PASSWORD')
    smtp_server = os.environ.get('MAIL_SMTP_SERVER')
    imap_server = os.environ.get('MAIL_IMAP_SERVER')

    print(f"Connecting to {imap_server} ...")
    # print(email_id, email_pass, smtp_server, imap_server)

    session = imaplib.IMAP4_SSL(imap_server)
    session.login(email_id, email_pass)
    print(f"Connected to {imap_server} . ")
    return session



def list_mailboxes(session):
    """
    Mailboxes are the folders in the email account.
    And comprise of default google mailboxes like 'INBOX', 'Sent', 'Drafts', 'Trash', 'Spam',
    and user created mailboxes(labels) .
    """
    result, mailboxes = session.list()
    return mailboxes


def validate_mailbox(mailbox, mailboxes):
    """
    Check if the mailbox exists in the list of mailboxes.
    """
    return any(mailbox in mb.decode() for mb in mailboxes)




def fetch_mails(session, limit=10, mail_folder='INBOX', fetch_recent=True, mailboxes=None, include_custom_labels=False):
    """
    Fetches emails from the specified mailbox
    :param session: IMAP session
    :param limit: Number of emails to fetch
    :param mail_folder: Mailbox to fetch emails from
    :param fetch_recent: Fetch the most recent emails
    :param mailboxes: List of mailboxes
    :param include_custom_labels: Include custom labels in the fetched emails
     - Whether the emails that have already been labelled by this service should be included in the fetched emails or not
     - This can happen when the few emails are categorized and placed in the same folder
     - occurs when remove-after-sort action is set of OFF)
    """
    # Fetch all mailboxes if not provided
    if mailboxes is None:
        mailboxes = list_mailboxes(session)

    source_folder_exists = validate_mailbox(mail_folder, mailboxes)
    if not source_folder_exists:
        print(f"   Mailbox '{mail_folder}' does not exist.")
        return

    # Format mailbox name if any special chars exist
    mail_folder = f'"{mail_folder}"'

    # Select the mailbox
    session.select(mail_folder)

    # Fetch email IDs
    status, messages = session.search(None, "ALL")
    email_ids = messages[0].split()

    # Order emails in reverse order if fetch_recent is True
    email_ids = email_ids[::-1] if fetch_recent else email_ids

    email_data = []
    i = 0
    with tqdm(total=min(limit, len(email_ids)), desc="Fetching Emails", ncols=100) as pbar:
        while limit > 0 and i < len(email_ids):
            email_id = email_ids[i]
            _, mail_flags = session.fetch(email_id, "(FLAGS)")
            _, mail_labels = session.fetch(email_id, "(X-GM-LABELS)")
            _, mail_uid = session.fetch(email_id, "(UID)")
            status, msg_data = session.fetch(email_id, "(RFC822)")

            # You can fetch FLAGS, X-GM-LABELS, UID in a single request
            # session.fetch(email_id, "(FLAGS X-GM-LABELS UID RFC822)")
            # output format [b'1011 (X-GM-LABELS ("@ Finances" "@ Verifications") UID 1157 FLAGS ())']
            # However parsing is a bit more complex so we fetch them separately

            # Parse the flags, labels and uid from the fetched data
            flags = parse_flags(mail_flags[0].decode('utf-8'))
            labels = parse_labels(mail_labels[0].decode('utf-8'))
            email_uid = parse_uid(mail_uid[0].decode('utf-8'))

            # Check if the mail is seen or unseen, In the below code when the imap reads the email it will mark it as seen
            # So we need to revert the status back to unseen if the mail was unseen
            mail_read_status = 'Unseen' if 'Seen' not in flags else 'Seen'

            # Verify if the mail has already been labelled in which case we wont label again
            already_labeled = any(label in labels for label in custom_labels)
            # If we are not including custom labels and the mail is already labelled, skip this mail
            if include_custom_labels and already_labeled:
                i += 1
                continue

            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_bytes(response_part[1])

                    # Decode subject
                    subject, encoding = decode_header(msg["Subject"])[0]

                    if isinstance(subject, bytes):
                        subject = subject.decode(encoding if encoding else "utf-8")

                    # Decode sender
                    from_ = msg.get("From")

                    body = ""
                    # Fetch email body
                    if msg.is_multipart():
                        for part in msg.walk():
                            content_type = part.get_content_type()
                            content_disposition = str(part.get("Content-Disposition"))
                            if "attachment" not in content_disposition:
                                payload = part.get_payload(decode=True)
                                if payload:  # Check if the payload is not None
                                    try:
                                        # Attempt to decode with detected encoding
                                        encoding = chardet.detect(payload)['encoding']
                                        body = payload.decode(encoding if encoding else 'utf-8', errors='replace')
                                    except Exception as e:
                                        body = ""
                                        print(f"   Unable to decode message: {str(e)}")
                                else:
                                    # NO Content in this part of the mail
                                    body = ""
                    else:
                        payload = msg.get_payload(decode=True)
                        if payload:  # Check if the payload is not None
                            try:
                                encoding = chardet.detect(payload)['encoding']
                                body = payload.decode(encoding if encoding else 'utf-8', errors='replace')
                            except Exception as e:
                                body = ""
                                print(f"   Unable to decode message: {str(e)}")
                        else:
                            # No Contentin this part of the email
                            body = ""

                    email_data.append({
                        "email_id": email_id.decode(),
                        'email_uid': email_uid,
                        "status": status,
                        "subject": subject,
                        "from": from_,
                        "raw_message": body,
                        "message": extract_text(body),
                        'read_status': mail_read_status

                    })

            # When in read the data any unseen mail will be set as seen so revert the status back to unseen
            if mail_read_status == 'Unseen':
                session.uid('STORE', email_uid, '-FLAGS', '(\Seen)')

            i += 1      # Move to the next email
            limit -= 1  # Decrement the limit as we have fetched one email
            pbar.update(1)

    # Convert email data to a Pandas DataFrame
    df = pd.DataFrame(email_data)
    return df


<a id = 'using-the-llm-for-email-categorization'></a>
### Using the LLM for Email Categorization

In this section, we'll leverage LangChain in combination with Ollama to create an LLM chain. This chain will use a carefully crafted prompt template and output parsing to categorize emails efficiently. The LLM will analyze the content of each email and assign it to the appropriate predefined category, enabling automated and accurate email management.

In [ ]:
import os

from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.llms import Ollama

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field


# Define the structure of out from llm
class MailType(BaseModel):
    category: str = Field(description="category of the given email")

# Return an langchain chain based on the chain type and model
def get_chain(chain_type, model):
    """
    Get the chain based on the chain type and model
    :param chain_type: str: Type of chain to use (openai/ollama)
    :param model: str: Model name to use (e.g., gpt-4o-mini)
      - If openai is selected as chain type ensure the OPENAI_API_KEY is set and the model is available
      - If ollama is selected as chain type ensure the ollama is installed and the model exists on your local machine
    """
    if chain_type == 'openai':
        # Make sure to set the OPENAI_API_KEY environment
        api_key = os.environ.get('OPENAI_API_KEY')
        llm = ChatOpenAI(
            api_key=api_key,
            model=model,  # eg gpt-4o-mini
        )
    elif chain_type == 'ollama':
        # Make sure Ollama is installed and the model is available
        llm = Ollama(
            model=model  # eg "llama3.1"
        )
    else:
        raise ValueError("Invalid chain type")

    # Define the system message for the model
    system_message = """
    You are an AI assistant that categorizes emails into predefined categories.
    You will be provided with the subject, sender, and content of an email, 
    along with a dictionary of categories. Based on this information, 
    determine the most appropriate category for the email.
    Important: 
    Please return only the category in the following JSON format: {{ "category": "category_name" }}.
    Do not include any explanation or additional text, only the JSON object.
    """

    # Define the human message template
    template = """
    Subject: `{subject}`
    Sender: `{sender}`
    Content:
    ```
    {content}
    ```
    Categories:
    ```
    {categories}
    ```
    """

    # Set up a parser + inject instructions into the prompt template.
    output_parser = JsonOutputParser(pydantic_object=MailType)

    # Create the chat prompt template
    prompt = ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template(system_message),
            HumanMessagePromptTemplate.from_template(template),
        ],
        input_variables=["subject", "sender", "content", "categories"],
        partial_variables={"format_instructions": output_parser.get_format_instructions()},
    )

    # Construct chain using prompt, llm and output_parser
    chain = prompt | llm | output_parser

    return chain


<a id = 'performing-actions-on-categorized-emails'></a>
### Performing Actions on Categorized Emails
Here are the functions that are used to move, delete and mark mails as read

In [ ]:
# Move a mail from source to target folder
def move_mail(session, source_folder, source_uid, target_folder, remove_after_moving, mailboxes=None):
    """
    Move a mail from the source folder to the target folder
    :param session: IMAP session
    :param source_folder:  The source folder where the mail is located
    :param source_uid: Mail UID
    :param target_folder: The target folder where the mail will be moved to
    :param remove_after_moving: Mark the mail as deleted in the source folder after moving
    :param mailboxes:
    :return:
    """
    # Fetch all mailboxes if not provided
    if mailboxes is None:
        mailboxes = list_mailboxes(session)

    # Check if the source and target folders exist
    source_folder_exists = validate_mailbox(source_folder, mailboxes)
    target_folder_exists = validate_mailbox(target_folder, mailboxes)
    # target_folder_exists = any(target_folder in mailbox.decode() for mailbox in mailboxes)

    if not source_folder_exists:
        print(f"   Source folder '{source_folder}' does not exist.")
        return False

    if not target_folder_exists:
        print(f"   Target folder '{target_folder}' does not exist.")
        return False

    # Format source and target folder names if any special chars exist
    source_folder = f'"{source_folder}"'
    target_folder = f'"{target_folder}"'

    # Select the source folder
    session.select(mailbox=source_folder, readonly=False)

    # If the target folder exists move the mail to the target folder
    result = session.uid('COPY', source_uid, target_folder)

    if result[0] == 'OK':
        print('   Mail copied to', target_folder)
        if remove_after_moving:
            # Mark the mail as deleted in the inbox
            session.uid('STORE', source_uid, '+FLAGS', '(\Deleted)')
            # Expunge the mailbox(delete the mail marked as deleted)
            session.expunge()
            print('   Mail removed from inbox')
    else:
        print('   Failed to move mail to', target_folder)


def move_to_bin(session, source_folder, source_uid, mailboxes=None):
    """
    Move a mail to the Bin
    """
    # Fetch all mailboxes if not provided
    if mailboxes is None:
        mailboxes = list_mailboxes(session)

    # Check if the source folder exists
    source_folder_exists = validate_mailbox(source_folder, mailboxes)
    if not source_folder_exists:
        print(f"   Source folder '{source_folder}' does not exist.")
        return False

    # Format source folder name if any special chars exist
    source_folder = f'"{source_folder}"'

    # Select the inbox or the current folder where the mail is located, make sure this folder exists
    status, data = session.select(source_folder)  # Change 'INBOX' to the appropriate mailbox if needed

    if status != 'OK':
        print('   Failed to select mailbox')
        return

    # Bin will always exist so , no need to verify its existence
    target_folder = '[Gmail]/Bin'  # Use '[Gmail]/Trash' for Gmail accounts

    # If the target folder exists move the mail to the target folder
    result = session.uid('COPY', source_uid, target_folder)

    if result[0] == 'OK':
        print('   Mail moved to Bin')
        # Mails copied to bin are automatically marked as deleted in the inbox and expunged
    else:
        print('   Failed to move mail to Bin')


def mark_as_read(session, source_folder, source_uid, mailboxes=None):
    """
    Mark a mail as read
    """
    # Fetch all mailboxes if not provided
    if mailboxes is None:
        mailboxes = list_mailboxes(session)

    # Check if the source folder exists
    source_folder_exists = validate_mailbox(source_folder, mailboxes)
    if not source_folder_exists:
        print(f"   Source folder '{source_folder}' does not exist.")
        return False

    # Format source folder name if any special chars exist
    source_folder = f'"{source_folder}"'

    # Select the inbox or the current folder where the mail is located, make sure this folder exists
    status, data = session.select(source_folder)

    if status != 'OK':
        print('   Failed to select mailbox')
        return

    # Mark the mail as read
    result = session.uid('STORE', source_uid, '+FLAGS', '(\Seen)')

    if result[0] == 'OK':
        print('   Mail marked as read')
    else:
        print('   Failed to mark mail as read')


def mark_as_unread(session, source_folder, source_uid, mailboxes=None):
    """
    Mark a mail as unread
    """
    # Fetch all mailboxes if not provided
    if mailboxes is None:
        mailboxes = list_mailboxes(session)

    # Check if the source folder exists
    source_folder_exists = validate_mailbox(source_folder, mailboxes)
    if not source_folder_exists:
        print(f"   Source folder '{source_folder}' does not exist.")
        return False

    # Format source folder name if any special chars exist
    source_folder = f'"{source_folder}"'

    # Select the inbox or the current folder where the mail is located, make sure this folder exists
    status, data = session.select(source_folder)

    if status != 'OK':
        print('   Failed to select mailbox')
        return

    # Mark the mail as unread
    result = session.uid('STORE', source_uid, '-FLAGS', '(\Seen)')

    if result[0] == 'OK':
        print('   Mail marked as unread')
    else:
        print('  Failed to mark mail as unread')


<a id = 'running-the-application'></a>
### Running the Application

Now that everything is set up, it's time to run the application. The below code will fetch emails, categorize them using the LLM, and perform the appropriate actions based on the predefined settings.

**Steps Involved:**  
1. **Connect to the Gmail account**: The application will establish a connection with your Gmail account.
2. **Fetch emails**: It will retrieve emails from the specified folder (e.g., `INBOX`).
3. **Categorize emails**: Each email will be processed through a language model to determine its category.
4. **Perform actions**: Based on the category, actions such as marking as read, moving to a folder, or deleting the email will be performed.
5. **Track performance**: The application will also track metrics like successful actions and failures.


In [ ]:
import time
import json


def main(source_folder='INBOX', limit=10, chain_type='ollama', model='llama3.1'):
    # Get Mail session
    session = mail_session()

    # Get all mailboxes
    mailboxes = list_mailboxes(session)

    # Consider only the first 1500 characters of the email content due to LLM Context limitations
    message_character_limit = 1500

    print(f'Fetching Mails from {source_folder}')
    mails_df = fetch_mails(session, limit=limit, mail_folder=source_folder, mailboxes=mailboxes)

    print(f"Loaded {len(mails_df)} emails from INBOX")
    mails_df['category'] = ""

    # Pick a model provider(openai/ollama) and model (make sure model exists(installed))
    # chain = get_chain(chain_type='openai', model='gpt-4o-mini')
    chain = get_chain(chain_type=chain_type, model=model)

    # Metrics to track service performance
    metrics = {
        "total": len(mails_df),
        "success": 0,
        "failed": 0,
        "llm-failures": 0,
        'action-failures': 0
    }

    # Have an LLM Categorize each email
    for index, row in mails_df.iterrows():
        start_time = time.time()
        subject = row['subject']
        sender = row['from']
        content = row['message'][:message_character_limit]
        print(f"-> Determining Category for : {row['email_uid']}")
        print(f"   Sender: {sender}")
        print(f"   Subject: {subject}")

        # Convert categories dictionary to a string format
        categories_str = json.dumps(categories)
        try:
            # Run the chain
            response = chain.invoke({
                "subject": subject,
                "sender": sender,
                "content": content,
                "categories": categories_str
            })

            print(f"   Time taken: {round(time.time() - start_time, 2)} Seconds")
            print(f"   Response: {response}")
            mails_df.at[index, 'category'] = response['category']
        except Exception as e:
            print(f"   LLM Error: {str(e)}")
            metrics['llm-failures'] += 1
            metrics['failed'] += 1
            continue

    # Go through each mail, Perform action based on mail category and its settings
    for index, row in mails_df.iterrows():
        print(f"-> Acting on mail : {row['email_uid']}, {row['subject']}, {row['category']}")

        category = row['category']
        if category not in categories.keys():
            print(f"   Invalid category: {category}")
            metrics['llm-failures'] += 1
            metrics['failed'] += 1
            continue

        # Get the action settings for the category
        settings = category_actions[category]

        try:
            # Perform reading action
            if settings['auto-read'] == 'ON':
                print('   Marking Mail as Read')
                mark_as_read(session, source_folder, row['email_uid'], mailboxes=mailboxes)

            # Perform deleting action
            if settings['auto-delete'] == 'ON':
                print('   Moving mail to bin')
                move_to_bin(session, source_folder, row['email_uid'], mailboxes=mailboxes)

            # Perform sorting/moving action
            elif settings['auto-sort'] == 'ON':
                target_folder = category_mapper[category]
                print(f"   Moving mail to {target_folder}")
                remove_after_sort = settings['remove-after-sort'] == 'ON'

                # If the target folder is same as source folder, do not move the mail
                if target_folder != source_folder:
                    move_mail(session, source_folder, row['email_uid'], target_folder, remove_after_sort, mailboxes)

        except Exception as e:
            print(str(e))
            metrics['failed'] += 1
            metrics['action-failures'] += 1

        else:
            metrics['success'] += 1

    session.close()
    print('Metrics', metrics)
    print('Category Count', mails_df['category'].value_counts().to_dict())


if __name__ == "__main__":
    main(
        source_folder='INBOX',  # Folder to fetch emails from
        limit=10,               # Number of emails to fetch
        chain_type='ollama',    # Model provider (openai/ollama)
        model='llama3.1'        # Model name
    )

<a id = 'conclusion-and-next-steps'></a>
### Conclusion and Next Steps

With this application, you've equipped yourself with a powerful tool to manage your Gmail inbox more effectively, using the latest advancements in language models for intelligent email categorization. This setup not only simplifies your inbox management but also allows for extensive customization through adjustable constants in the `constants.py` file.

**Key Takeaways:**  
- **Application Logs**: Throughout its execution, the application provides detailed logs of each significant action, such as categorizing emails, applying actions (like sorting or deleting), and handling any errors. These logs are invaluable for troubleshooting and ensuring the application functions as expected.
  
- **Metrics Summary**: After processing, the application outputs a summary of its performance, displaying key metrics including:
  ```python
  metrics = {
      'total': 100,          # Total number of emails processed
      'success': 100,        # Number of emails successfully categorized and processed
      'failed': 0,           # Number of emails that could not be processed
      'llm-failures': 0,     # Number of failures related to the language model
      'action-failures': 0   # Number of failures related to actions like sorting or deleting emails
  }
  ```

- **Category Count**: Additionally, the application provides a breakdown of how your emails were distributed across different categories:
  ```python
  category_count = {
      'Promotions': 58, 
      'Work': 16, 
      'Verifications': 13, 
      'Spam': 8, 
      'Finance Promotions': 1, 
      'Finances': 1, 
      'Personal': 1, 
      'Reminders': 1, 
      'Support': 1
  }
  ```
  This count offers a clear view of how your emails were categorized, helping you understand the organization process.

By regularly reviewing these logs and metrics, you can ensure that your Gmail inbox is being managed as intended, with all emails correctly categorized and processed according to your preferences.

**Next Steps**  
You've taken a significant step toward better inbox management by integrating LLMs for intelligent email categorization. To further enhance performance, consider fine-tuning the constants in the `constants.py` file, such as category definitions and actions, to better align with your email habits. However, it's important to be aware of the potential for false categorization. Regularly reviewing and adjusting your settings will help minimize this risk and ensure that your emails are sorted accurately.

With this tool, your Gmail inbox is now more organized, efficient, and tailored to your needs.